In [7]:
pip install xmltodict

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\LG\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [17]:
pip install feedparser

     ---------------------------------------- 81.1/81.1 KB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for sgmllib3k, since package 'wheel' is not installed.
  Running setup.py install for sgmllib3k: started
  Running setup.py install for sgmllib3k: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\LG\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [15]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\LG\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import urllib.robotparser
import xmltodict
import requests
import feedparser
import os
import requests
import re
from bs4 import BeautifulSoup
from dateutil import parser

### 로이터 뉴스 

### URL 생성하기

In [13]:
# robots.txt해석
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.reuters.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.reuters.com/sitemap.xml')

True

In [14]:
rp.can_fetch('*', 'https:/www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml')

True

In [15]:
rp.can_fetch("*", "https://www.reuters.com/finance/stocks/option")

False

#### sitemap.xml 파일로 URL 획득

In [16]:
sitemap = xmltodict.parse(requests.get(
    'https://www.reuters.com/arc/outboundfeeds/news-sitemap/?outputType=xml').text
)

In [17]:
# 파일을 다운로드 하기 전에 dict에 무엇이 있는지 확인해보기
urls = [url['loc'] for url in sitemap['urlset']['url']]
# 공간을 차지하지 않도록 맨 앞 URL 몇 개만 출력한다.
print("\n".join(urls[0:3]))

https://www.reuters.com/markets/us/blackstone-reit-says-redemption-requests-declined-june-2023-07-03/
https://www.reuters.com/world/middle-east/dubai-leads-most-gulf-markets-higher-qatar-falls-2023-07-03/
https://www.reuters.com/fr/actualites/ukraine-un-ancien-haut-responsable-de-la-scurit-accus-de-trahison-2023-07-03/


#### RSS에서 URL 획득

In [18]:
import feedparser
feed = feedparser.parse('http://web.archive.org/web/20200613003232if_/http://feeds.reuters.com/Reuters/worldNews')

In [19]:
[(e.title, e.link) for e in feed.entries]

[('Mexico City to begin gradual exit from lockdown on Monday',
  'http://feeds.reuters.com/~r/Reuters/worldNews/~3/OQtkVdAqHos/mexico-city-to-begin-gradual-exit-from-lockdown-on-monday-idUSKBN23K00R'),
 ('Mexico reports record tally of 5,222 new coronavirus cases',
  'http://feeds.reuters.com/~r/Reuters/worldNews/~3/Rkz9j2G7lJU/mexico-reports-record-tally-of-5222-new-coronavirus-cases-idUSKBN23K00B'),
 ('Venezuela supreme court to swear in new electoral council leaders, government says',
  'http://feeds.reuters.com/~r/Reuters/worldNews/~3/cc3R5aq4Ksk/venezuela-supreme-court-to-swear-in-new-electoral-council-leaders-government-says-idUSKBN23J39T'),
 ("One-fifth of Britain's coronavirus patients were infected in hospitals: Telegraph",
  'http://feeds.reuters.com/~r/Reuters/worldNews/~3/1_7Wb0S_6-8/one-fifth-of-britains-coronavirus-patients-were-infected-in-hospitals-telegraph-idUSKBN23J382'),
 ('France to lift border controls for EU travellers on June 15',
  'http://feeds.reuters.com/~r/

In [20]:
# 실제 URL 뽑아보기
[e.id for e in feed.entries]

['https://www.reuters.com/article/us-health-coronavirus-mexico-city/mexico-city-to-begin-gradual-exit-from-lockdown-on-monday-idUSKBN23K00R?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-health-coronavirus-mexico/mexico-reports-record-tally-of-5222-new-coronavirus-cases-idUSKBN23K00B?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-venezuela-politics/venezuela-supreme-court-to-swear-in-new-electoral-council-leaders-government-says-idUSKBN23J39T?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-health-coronavirus-britain-hospitals/one-fifth-of-britains-coronavirus-patients-were-infected-in-hospitals-telegraph-idUSKBN23J382?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-health-coronavirus-france-borders/france-to-lift-border-controls-for-eu-travellers-on-june-15-idUSKBN23J385?feedType=RSS&feedName=worldNews',
 'https://www.reuters.com/article/us-health-coronavirus-brazil/brazils-covid-19-deaths-sur

### 데이터 다운로드

### 파이썬을 사용한 HTML 페이지 다운로드

In [21]:
%%time
output_directory = './data/'
s = requests.Session()
for url in urls[0:10] : 
    # URL의 마지막/뒤 부분을 가져와서 파일 이름으로 사용한다.
    file = url.split("/")[-1]
    # file_path = os.path.join(output_directory, file)
    
    r = s.get(url)
    if r.ok : 
        with open('./file', 'w+b') as f : 
            f.write(r.text.encode('utf-8'))
    else : 
        print('error with URL %s' % url)

CPU times: total: 15.6 ms
Wall time: 8.32 s


#### wget를 사용한 HTML 페이지 다운로드

In [22]:
with open('urls.txt', 'w+b') as f : 
    f.write('\n'.join(urls).encode('utf-8'))

### 반정형 데이터 추출

#### 정규 표현식을 사용한 추출

In [23]:
url = 'https://www.reuters.com/article/us-health-vaping-marijuana-idUSKBN1WG4KT'

# 마지막 / 뒤의 부분을 파일 이름으로 사용하기
file = url.split('/')[-1] + '.html'
r = requests.get(url)
with open(file, 'w+b') as f : 
    f.write(r.text.encode('utf-8'))

In [25]:
with open(file, "r", encoding='utf-8') as f:
    html = f.read()
    g = re.search(r'<title>(.*)</title>', html, re.MULTILINE|re.DOTALL)
    if g:
        print(g.groups()[0])

Banned in Boston: Without vaping, medical marijuana patients must adapt | Reuters


#### HTML 파서를 사용한 데이터 추출

In [26]:
WA_PREFIX = 'http://web.archive.org/web/20200118131624/'
html = s.get(WA_PREFIX + url).text

In [27]:
soup = BeautifulSoup(html, 'html.parser')
soup.select('h1.ArticleHeader_headline')

[<h1 class="ArticleHeader_headline">Banned in Boston: Without vaping, medical marijuana patients must adapt</h1>]

In [28]:
soup.h1

<h1 class="ArticleHeader_headline">Banned in Boston: Without vaping, medical marijuana patients must adapt</h1>

In [29]:
soup.h1.text

'Banned in Boston: Without vaping, medical marijuana patients must adapt'

In [30]:
soup.title.text

'\n                Banned in Boston: Without vaping, medical marijuana patients must adapt - Reuters'

##### 기사 텍스트 추출

In [31]:
soup.select_one('div.StandardArticleBody_body').text

'BOSTON (Reuters) - In the first few days of the four-month ban on all vaping products in Massachusetts, Laura Lee Medeiros, a medical marijuana patient, began to worry.\xa0 FILE PHOTO: An employee puts down an eighth of an ounce marijuana after letting a customer smell it outside the Magnolia cannabis lounge in Oakland, California, U.S. April 20, 2018. REUTERS/Elijah NouvelageThe 32-year-old massage therapist has a diagnosis of post-traumatic stress disorder (PTSD) from childhood trauma. To temper her unpredictable panic attacks, she relied on a vape pen and cartridges filled with the marijuana derivatives THC and CBD from state dispensaries. There are other ways to get the desired effect from  marijuana, and patients have filled dispensaries across the state in recent days to ask about edible or smokeable forms. But Medeiros has come to depend on her battery-powered pen, and wondered how she would cope without her usual supply of cartridges.  “In the midst of something where I’m on t

##### 이미지 캡션 추출

In [32]:
soup.select('div.StandardArticleBody_body figure img')

[<img aria-label="FILE PHOTO: An employee puts down an eighth of an ounce marijuana after letting a customer smell it outside the Magnolia cannabis lounge in Oakland, California, U.S. April 20, 2018. REUTERS/Elijah Nouvelage" src="//web.archive.org/web/20200118131624im_/https://s3.reutersmedia.net/resources/r/?m=02&amp;d=20191001&amp;t=2&amp;i=1435991144&amp;r=LYNXMPEF9039L&amp;w=20"/>,
 <img src="//web.archive.org/web/20200118131624im_/https://s3.reutersmedia.net/resources/r/?m=02&amp;d=20191001&amp;t=2&amp;i=1435991145&amp;r=LYNXMPEF9039M"/>]

In [33]:
# 이미지의 캡션에 관심이 있는 경우
soup.select('div.StandardArticleBody_body figcaption')

[<figcaption><div class="Image_caption"><span>FILE PHOTO: An employee puts down an eighth of an ounce marijuana after letting a customer smell it outside the Magnolia cannabis lounge in Oakland, California, U.S. April 20, 2018. REUTERS/Elijah Nouvelage</span></div></figcaption>,
 <figcaption class="Slideshow_caption">Slideshow<span class="Slideshow_count"> (2 Images)</span></figcaption>]

##### URL 추출

In [34]:
# 표준 URL : canonical
soup.find('link', {'rel':'canonical'})['href']

'http://web.archive.org/web/20200118131624/https://www.reuters.com/article/us-health-vaping-marijuana-idUSKBN1WG4KT'

##### 목록 정보(작성자) 추출

In [35]:
soup.find('meta', {'name':'Author'})['content']

'Jacqueline Tempera'

In [36]:
# 모든 작성자 추출하기
sel = "div.BylineBar_first-container.ArticleHeader_byline-bar div.BylineBar_byline span"
soup.select(sel)

[<span><a href="/web/20200118131624/https://www.reuters.com/journalists/jacqueline-tempera" target="_blank">Jacqueline Tempera</a>, </span>,
 <span><a href="/web/20200118131624/https://www.reuters.com/journalists/jonathan-allen" target="_blank">Jonathan Allen</a></span>]

In [37]:
# 작성자 이름만 추출
[a.text for a in soup.select(sel)]

['Jacqueline Tempera, ', 'Jonathan Allen']

##### 링크 텍스트 추출(섹션)

In [38]:
soup.select_one('div.ArticleHeader_channel a').text

'Health News'

##### 읽기 시간 추출

In [39]:
soup.select_one('p.BylineBar_reading-time').text

'6 Min Read'

##### ID속성 추출

In [41]:
soup.select_one('div.StandardArticle_inner-container')['id']

'USKBN1WG4KT'

##### 속성 추출
- 속성 정보 : 텍스트 끝에서 찾을 수 있음.

In [42]:
soup.select_one('p.Attribution_content').text

'Reporting Jacqueline Tempera in Brookline and Boston, Massachusetts, and Jonathan Allen in New York; Editing by Frank McGurty and Bill Berkrot'

##### 타임스탬프 추출

In [44]:
ptime = soup.find('meta', {'property' : 'og:article:published_time'})['content']
print(ptime)

2019-10-01T19:23:16+0000


In [47]:
# datetime객체로 변환하기
parser.parse(ptime)

datetime.datetime(2019, 10, 1, 19, 23, 16, tzinfo=tzutc())

#### 스파이더링(웹 크롤링)